## Interval Analysis

In [2]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=-1) # added softmax for probabilities
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()


ModuleNotFoundError: No module named 'torch'

In [ ]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [ ]:
train_model(model, 10)
test_model(model)

### Write the interval analysis for the simple model

In [ ]:
# Install bound_propagation if needed
# !pip install bound_propagation

import bound_propagation as bp
from bound_propagation import IntervalBounds, HyperRectangle

def interval_analysis(model, x, epsilon):
    """
    Perform interval analysis on the model for a given input and epsilon perturbation.
    
    Args:
        model: The neural network model
        x: Input tensor (single image or batch)
        epsilon: Maximum perturbation magnitude
    
    Returns:
        bounds: IntervalBounds object containing lower and upper bounds for the output
    """
    # Create input bounds: [x - epsilon, x + epsilon]
    # Clamp to valid pixel range [0, 1]
    x_lower = torch.clamp(x - epsilon, 0, 1)
    x_upper = torch.clamp(x + epsilon, 0, 1)
    
    # Create HyperRectangle for the input bounds
    input_bounds = HyperRectangle(x_lower, x_upper)
    
    # Create interval bounds from the input
    bounds = IntervalBounds(model, input_bounds)
    
    # Propagate bounds through the network
    output_bounds = bounds.concretize()
    
    return output_bounds

def verify_robustness(model, x, true_label, epsilon):
    """
    Verify if the model is robust to epsilon perturbations for a given input.
    
    Args:
        model: The neural network model
        x: Input tensor (single image)
        true_label: True class label
        epsilon: Maximum perturbation magnitude
    
    Returns:
        is_robust: Boolean indicating if the model is provably robust
        output_bounds: The output bounds from interval analysis
    """
    model.eval()
    
    # Get interval bounds for the output
    output_bounds = interval_analysis(model, x, epsilon)
    
    # Extract lower and upper bounds
    lower_bounds = output_bounds.lower
    upper_bounds = output_bounds.upper
    
    # Check if true label has minimum lower bound that is greater than
    # maximum upper bound of all other labels
    is_robust = True
    true_label_lower = lower_bounds[0, true_label]
    
    for i in range(lower_bounds.shape[1]):
        if i != true_label:
            if upper_bounds[0, i] >= true_label_lower:
                is_robust = False
                break
    
    return is_robust, output_bounds

# Test interval analysis on a single image
model.eval()
test_iter = iter(test_loader)
test_images, test_labels = next(test_iter)
test_images, test_labels = test_images.to(device), test_labels.to(device)

# Select first image
x_test = test_images[0:1]
y_test = test_labels[0]

# Test with different epsilon values
epsilons = [0.01, 0.05, 0.1, 0.2, 0.3]

print(f"Testing interval analysis for image with true label: {y_test.item()}")
print("-" * 60)

for eps in epsilons:
    is_robust, bounds = verify_robustness(model, x_test, y_test.item(), eps)
    print(f"Epsilon: {eps:.2f}, Provably Robust: {is_robust}")
    
# Verify robustness on a batch of test images
def batch_verify_robustness(model, test_loader, epsilon, num_samples=100):
    """
    Verify robustness on a batch of test images.
    
    Args:
        model: The neural network model
        test_loader: DataLoader for test data
        epsilon: Maximum perturbation magnitude
        num_samples: Number of samples to verify
    
    Returns:
        robust_count: Number of provably robust samples
        total_count: Total number of samples verified
        robust_accuracy: Percentage of provably robust samples
    """
    model.eval()
    robust_count = 0
    total_count = 0
    
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        for i in range(images.shape[0]):
            if total_count >= num_samples:
                break
                
            x = images[i:i+1]
            y = labels[i].item()
            
            # Get model prediction
            with torch.no_grad():
                pred = model(x).argmax(dim=1).item()
            
            # Only verify correctly classified samples
            if pred == y:
                is_robust, _ = verify_robustness(model, x, y, epsilon)
                if is_robust:
                    robust_count += 1
                total_count += 1
        
        if total_count >= num_samples:
            break
    
    robust_accuracy = 100 * robust_count / total_count if total_count > 0 else 0
    return robust_count, total_count, robust_accuracy

# Verify robustness on multiple samples
print("\n" + "=" * 60)
print("Batch verification of robustness")
print("=" * 60)

for eps in [0.01, 0.05, 0.1]:
    robust_count, total_count, robust_acc = batch_verify_robustness(
        model, test_loader, eps, num_samples=50
    )
    print(f"Epsilon: {eps:.2f}, Robust: {robust_count}/{total_count}, "
          f"Robust Accuracy: {robust_acc:.2f}%")